In [1]:
import numpy as np
import os
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier  
os.chdir("C:/Users/ekorkmaz/Desktop/EnkTelefonYedek/kodluyoruz/sonuncu_ödeb_hafta_5") 

In [64]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [10]:
from sklearn.tree import DecisionTreeClassifier

# 1-) Decision Tree

In [13]:
cart=DecisionTreeClassifier()#hedef değişken kategorik
cart_model=cart.fit(X_train,y_train)
cart_model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [28]:
from skompiler import skompile

#print(skompile(cart_model.predict).to("python/code"))

In [15]:
y_pred=cart_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9306930693069307

In [16]:
print(classification_report(y_test,y_pred))#default parametrelerle kurulan modelimizin basarisini inceleriz;

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       554
           1       0.62      0.50      0.55        52

   micro avg       0.93      0.93      0.93       606
   macro avg       0.79      0.74      0.76       606
weighted avg       0.93      0.93      0.93       606



# 1.1) Decision Tree Model Tuning

In [17]:
?cart_model

In [18]:
cart_grid={"max_depth":range(1,10),"min_samples_split":list(range(2,50))}
#max_Depth:ne kadar derinlikte olsun
#min_samples_split:yapragı bölerken en az kaça böliyim datayi bilgisidir.

In [19]:
cart=tree.DecisionTreeClassifier()
cart_cv=GridSearchCV(cart,cart_grid,cv=10,n_jobs=-1,verbose=2)
cart_cv_model=cart_cv.fit(X_train,y_train)

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:   17.4s finished


In [20]:
print("EN iyi parametreler: "+str(cart_cv_model.best_params_))

EN iyi parametreler: {'max_depth': 8, 'min_samples_split': 2}


In [23]:
#simdi en iyi parametrelerle tekrar model kurariz ve default degerler ile olan accuracy farklarına bakarız
cart=tree.DecisionTreeClassifier(max_depth=8,min_samples_split=2)
cart_tuned=cart.fit(X_train,y_train)

In [24]:
y_pred=cart_tuned.predict(X_test)#kurdugumuz model ile tahmin edilen y leri y_prede atadık
accuracy_score(y_test,y_pred)#gerçek y degerleri ile modelimizin tahmin ettiği y değerlerini kıyaslayarak accuracyi buluruz

0.9455445544554455

In [25]:
print(classification_report(y_test,y_pred))
#en iyi parametrelerle kurulan modelimizin basarisini inceleriz;
#0 lar icin recall ve f1 score iyilesmis
#1ler icin precision ve f1 score degeri ciddi oranda iyilesmistir

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.81      0.48      0.60        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.88      0.73      0.79       606
weighted avg       0.94      0.95      0.94       606



# 2-) Random Forest

In [63]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
rf_model=RandomForestClassifier().fit(X_train,y_train)#default degerlerle modelimizi kurduk

C:\Users\ekorkmaz\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [30]:
y_pred=rf_model.predict(X_test) # kurdugumuz modelin tahmin ettigi y'lerimizi buluruz
accuracy_score(y_test,y_pred)#gercek y degerleri ile modelimizle tahmin edilen y lerden accuracyi buluruz

0.9504950495049505

In [31]:
print(classification_report(y_test,y_pred)) #default degerlerle kurulan modelimizin basarisini inceleriz

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       554
           1       0.96      0.44      0.61        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.95      0.72      0.79       606
weighted avg       0.95      0.95      0.94       606



# 2.1-) Random Forest Model Tuning 

In [33]:
?RandomForestClassifier

In [34]:
rf_params={"max_depth":[2,5,8,10],"max_features":[2,5,8],"n_estimators":[10,500,1000],"min_samples_split":[2,5,10]}
#GridSearchde kullancagımız dictionary yaptık

In [36]:
rf_model=RandomForestClassifier()
rf_cv_model=GridSearchCV(rf_model,rf_params,cv=10,n_jobs=-1,verbose=1)

In [37]:
rf_cv_model.fit(X_train,y_train)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   40.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  6.5min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [2, 5, 8, 10], 'max_features': [2, 5, 8], 'n_estimators': [10, 500, 1000], 'min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [38]:
print("En iyi parametreler: "+str(rf_cv_model.best_params_))#random olduğu için paramlar farklı çıkabilir yani her çalıştırdığında

En iyi parametreler: {'max_depth': 10, 'max_features': 5, 'min_samples_split': 10, 'n_estimators': 10}


In [39]:
#simdi en iyi parametrelerle tekrar model kurariz ve default degerler ile olan accuracy farklarına bakarız
rf_tuned=RandomForestClassifier(max_depth=10,max_features=5,min_samples_split=10,n_estimators=10)

In [40]:
rf_tuned.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [42]:
y_pred=rf_tuned.predict(X_test) # kurdugumuz modelin tahmin ettigi y'lerimizi buluruz
accuracy_score(y_test,y_pred)#gercek y degerleri ile modelimizle tahmin edilen y lerden accuracyi buluruz


0.9455445544554455

In [44]:
print(classification_report(y_test,y_pred))
#en iyi parametrelerle kurulan modelimizin basarisini inceleriz;
#0 lar icin precision düşmüş
#1ler icin precision artarken  recall ve f1 score degeri düşmüştür.
#Bu durumda default degerler ile kurulan modelin daha basarili olmasinin sebebi data seçerken ki randomluktan kaynaklanıyor olabilir

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       554
           1       1.00      0.37      0.54        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.97      0.68      0.75       606
weighted avg       0.95      0.95      0.93       606



# 3-)Gradient Boosting

#Normal hızdaki boosting;

In [ ]:
#Datamızdaki 0ların ve 1lerin oranı unbalaced ise boosting uygulamaları daha uygundur çünkü bu modellerde 
#daha iyi öğrendiği sampleları seçmeye eğilimlidir.

In [62]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [48]:

from sklearn.ensemble import GradientBoostingClassifier

In [46]:
gbm_model=GradientBoostingClassifier().fit(X_train,y_train) #default degerlerle model kurduk

In [49]:
y_pred=gbm_model.predict(X_test)#default degerlerden kurulan modelden tahmin edilen y lerimizi aldık
accuracy_score(y_test,y_pred) #gercek y degerleri ile modelimizin tahmin ettigi y degerlerinin farkından accuracyi bulduk

0.9455445544554455

In [50]:
print(classification_report(y_test,y_pred)) #default degerlerle kurulan modelimizin basarisini inceleriz

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.85      0.44      0.58        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.90      0.72      0.78       606
weighted avg       0.94      0.95      0.94       606



# 3.1-) Gradient Boosting Model Tuning

In [52]:
gbm_params={"learning_rate":[0.001,0.01,0.1,0.05],"n_estimators":[10,50,100],"max_depth":[3,5,10],"min_samples_split":[2,5,10]}

In [53]:
gbm=GradientBoostingClassifier()
gbm_cv=GridSearchCV(gbm,gbm_params,cv=10,n_jobs=-1,verbose=5)

In [54]:
gbm_cv.fit(X_train,y_train)


Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  87 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 267 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 519 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 801 tasks      | elapsed:   47.0s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  1.1min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.05], 'n_estimators': [10, 50, 100], 'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [55]:
print("En iyi parametreler: "+str(gbm_cv.best_params_))

En iyi parametreler: {'learning_rate': 0.1, 'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 50}


In [56]:
gbm=GradientBoostingClassifier(learning_rate=0.1,max_depth=10,min_samples_split=5,n_estimators=50)

In [57]:
gbm_tuned=gbm.fit(X_train,y_train) #en iyi parametrelerle modelimizi tekrar kurduk

In [58]:
y_pred=gbm_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

0.9537953795379538

In [59]:
print(classification_report(y_test,y_pred)) #en iyi degerlerle kurulan modelimizin basarisini inceleriz
#0 lar icin; precision ve f1 degeri iyilesmis
#1 ler icin ;tüm degerler iyilesmis

              precision    recall  f1-score   support

           0       0.96      0.99      0.98       554
           1       0.90      0.52      0.66        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.93      0.76      0.82       606
weighted avg       0.95      0.95      0.95       606



# 4-) XGBOOST

In [61]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [65]:
from xgboost import XGBClassifier

In [66]:
xgb_model=XGBClassifier().fit(X_train,y_train)

In [67]:
y_pred=gbm_model.predict(X_test)#default degerlerden kurulan modelden tahmin edilen y lerimizi aldık
accuracy_score(y_test,y_pred) #gercek y degerleri ile modelimizin tahmin ettigi y degerlerinin farkından accuracyi bulduk

0.9455445544554455

In [68]:
print(classification_report(y_test,y_pred)) #default degerlerle kurulan modelimizin basarisini inceleriz

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.85      0.44      0.58        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.90      0.72      0.78       606
weighted avg       0.94      0.95      0.94       606



# 4.1-) XGBOOST Model Tuning

In [73]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [74]:
xgb_params={
    'n_estimators':[10,50,100,200],
    'subsample':[0.6,0.8,1.0],
    'max_depth':[3,4,5,6],
    'learning_rate':[0.1,0.01,0.02,0.05],
    'min_samples_split':[2,5,10]
}
#sub sample =agac olustururken traine ne kadar veri cekiyim,bilgisidir. 
#datamın farklı parametrelerle model kurması icin dictionary olusturduk
#overfitting olmamması icin cok fazla derinlikte agac olusturmasak iyi olur cunku her yeni yaprak olusturumunda adtayı 
#böldugu icin yapraga az veri kaliyor onları ezberlememesi icin optimum derinlikte agac kurdurmalıyız 

In [75]:
xgb=XGBClassifier()

In [76]:
xgb_cv_model=GridSearchCV(xgb,xgb_params,cv=10,n_jobs=-1,verbose=1) 
#cv=cross validation yani yukardaki parametreler ısıgında 10 tane farklı agac kurar(10 tane farklı sample cekerek)

In [77]:
xgb_cv_model.fit(X_train,y_train)
#modelimize datamızı veririz(fit ederiz) 

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 384 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 867 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1327 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2162 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2902 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3851 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 4993 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:  8.5min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100, 200], 'subsample': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5, 6], 'learning_rate': [0.1, 0.01, 0.02, 0.05], 'min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [78]:
xgb_cv_model.best_params_

{'learning_rate': 0.1,
 'max_depth': 4,
 'min_samples_split': 2,
 'n_estimators': 200,
 'subsample': 0.8}

In [79]:
xgb=XGBClassifier(learning_rate=0.01,
                 max_depth=4,
                  min_samples_split=2,
                  n_estimators=200,
                  subsample=0.6
                 )

In [80]:
xgb_tuned=xgb.fit(X_train,y_train)

In [81]:
y_pred=xgb_tuned.predict(X_test)
accuracy_score(y_test,y_pred)

0.9488448844884488

In [82]:
print(classification_report(y_test,y_pred))#xgboostun best parametrelerle kurdugu modelin accuracysini yorumlarız 
# 0'lar için;recall degeri iyilesmis
# 1'ler için;precision iyilesmis recall kötülesmis

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       554
           1       1.00      0.40      0.58        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.97      0.70      0.77       606
weighted avg       0.95      0.95      0.94       606



# 5-) LIGHTGBM 

#light Boosting

In [84]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [85]:
from lightgbm import LGBMClassifier

In [86]:
lgbm_model=LGBMClassifier().fit(X_train,y_train)


In [87]:
y_pred=lgbm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9537953795379538

In [88]:
print(classification_report(y_test,y_pred))#lightgbm'nindefault parametrelerle kurdugu modelin accuracysi

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       554
           1       0.96      0.48      0.64        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.96      0.74      0.81       606
weighted avg       0.95      0.95      0.95       606



# 5.1-) LıghtGBM Model Tuning

In [94]:
?lgbm_model  
#LGBM parametreleri hakkında bilgi verir;

In [95]:
lgbm_params={
    'n_estimators':[10,50,100,200],
    'subsample':[0.6,0.8,1.0],
    'max_depth':[3,4,5,6],
    'learning_rate':[0.1,0.01,0.02,0.05],
    "min_child":[5,10,20]
}

In [96]:
lgbm=LGBMClassifier()
lgbm_cv_model=GridSearchCV(lgbm,lgbm_params,
                          cv=10,
                          n_jobs=-1,
                          verbose=1)

In [97]:
lgbm_cv_model.fit(X_train,y_train)#modeli datamızla  eğitiyoruz ,kuruyoruz

Fitting 10 folds for each of 576 candidates, totalling 5760 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  59 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 659 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 1573 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 2273 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 3173 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4273 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 5573 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 5753 out of 5760 | elapsed:  3.2min remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 5760 out of 5760 | elapsed:  3.2min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 50, 100, 200], 'subsample': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5, 6], 'learning_rate': [0.1, 0.01, 0.02, 0.05], 'min_child': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [98]:
lgbm_cv_model.best_params_#subsample;sample cekerken datanın yüzde kacini kullaniyim bilgisidir

{'learning_rate': 0.1,
 'max_depth': 3,
 'min_child': 5,
 'n_estimators': 200,
 'subsample': 0.6}

In [99]:
lgbm=LGBMClassifier(learning_rate=0.1,
                 max_depth=3,
                  min_child=5,#en alttaki yaprakta kac tane veri kalsın bilgisidir.
                  n_estimators=200,
                  subsample=0.6
                 )

In [100]:
lgbm_tuned=lgbm.fit(X_train,y_train)
y_pred=lgbm_tuned.predict(X_test)


In [101]:
accuracy_score(y_test,y_pred)

0.9488448844884488

In [102]:
print(classification_report(y_test,y_pred))#boostun best parametrelerle kurdugu modelin accuracysini yorumlarız 
# 0 lar için f1 lerin degeri düşmüş 
# 1 ler için precision ve recall degerleri düşmüş f1 degeri iyilesmis

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       554
           1       0.92      0.44      0.60        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.94      0.72      0.79       606
weighted avg       0.95      0.95      0.94       606



# 6-) Neural Network

In [4]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler=StandardScaler()

In [7]:
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test) #aralarındaki ilişki lineer oldugu düşünüldüğü için normalize ederiz veriyi

C:\Users\ekorkmaz\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ekorkmaz\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\ekorkmaz\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
X_test_scaled[0:5]

array([[ 0.66119779,  0.41857973,  0.45688065, -1.1976411 ,  1.5763105 ,
        -0.32687081,  0.08335509,  0.56846498,  1.52185726,  0.11670154,
         0.6374751 , -0.6374751 , -0.40385294, -0.43767703, -0.80952709,
         1.69994328, -0.11678881, -0.18350739],
       [ 0.20052987,  0.76904412,  1.29483717, -0.4082825 , -0.26772465,
        -0.32687081,  0.38153842,  1.79767255,  0.15796644,  1.03330214,
         0.6374751 , -0.6374751 , -0.40385294,  2.28478978, -0.80952709,
        -0.58825492, -0.11678881, -0.18350739],
       [-0.19690911,  1.05263158,  0.975908  , -0.67140204, -0.26772465,
        -0.32687081, -0.28954975, -0.6607426 , -0.1567776 , -0.97373513,
         0.6374751 , -0.6374751 , -0.40385294,  2.28478978, -0.80952709,
        -0.58825492, -0.11678881, -0.18350739],
       [-0.26917075, -1.19611852,  1.06360535, -0.01360321, -0.26772465,
        -0.32687081,  1.45586509, -0.6607426 ,  0.47271047, -0.76194009,
        -1.56868872,  1.56868872, -0.40385294, -0.437

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
mlpc=MLPClassifier().fit(X_train_scaled,y_train)

C:\Users\ekorkmaz\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [11]:
y_pred=mlpc.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.9438943894389439

In [12]:
print(classification_report(y_test,y_pred)) 
#default parametrelerle kurdugumuz nn modeli

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       554
           1       0.88      0.40      0.55        52

   micro avg       0.94      0.94      0.94       606
   macro avg       0.91      0.70      0.76       606
weighted avg       0.94      0.94      0.93       606



# 6.1-) Model Tuning

In [14]:
mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.001, 0.005,0.0001],
              "hidden_layer_sizes": [(10,10,10),#2 layer olsun her layerda max 10 node olsun 
                                     (100,100),
                                     (100,100,100),#2 layer olsun her layerda max 100 node olsun 
                                     (3,5), #single layer
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],
              "activation": ["relu","logistic"]}
#alpha :duracagi error sınırı bilgisidir.

In [16]:
mlpc=MLPClassifier()
mlpc_cv_model=GridSearchCV(mlpc,mlpc_params,
                          cv=10,
                          n_jobs=-1,
                          verbose=1)
mlpc_cv_model.fit(X_train_scaled,y_train)#modelimizi kurarız

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 13.1min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 18.4min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [0.1, 0.01, 0.02, 0.001, 0.005, 0.0001], 'hidden_layer_sizes': [(10, 10, 10), (100, 100), (100, 100, 100), (3, 5), (5, 3)], 'solver': ['lbfgs', 'adam', 'sgd'], 'activation': ['relu', 'logistic']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [17]:
print("En iyi parametreler:" + str(mlpc_cv_model.best_params_))

En iyi parametreler:{'activation': 'relu', 'alpha': 0.1, 'hidden_layer_sizes': (100, 100), 'solver': 'lbfgs'}


In [20]:
mlpc_tuned=MLPClassifier(activation="relu",
                        alpha=0.1,
                        hidden_layer_sizes=(100,100),
                         solver="lbfgs"
                        )

In [22]:
mlpc_tuned.fit(X_train_scaled,y_train)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [24]:
y_pred=mlpc_tuned.predict(X_test_scaled)
accuracy_score(y_test,y_pred)


0.9504950495049505

In [25]:
print(classification_report(y_test,y_pred))
#en iyi parametrelerle NN modeli kurdugumuzda;
# 0 lar icin ; precision degerimiz iyileşmiş
# 1 ler icin;recall ve f1 iyilesirken precisionımız düşmüş
#accuracymiz artmis

              precision    recall  f1-score   support

           0       0.96      0.99      0.97       554
           1       0.81      0.56      0.66        52

   micro avg       0.95      0.95      0.95       606
   macro avg       0.88      0.77      0.82       606
weighted avg       0.95      0.95      0.95       606



# 7-) SVM

In [3]:
data=pd.read_csv("hmelq.csv")
df=data.copy()
df=df.dropna()
y=df["bad"]
X=df.drop(["bad"],axis=1)
X=pd.get_dummies(X)
X=pd.DataFrame(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [4]:
svm_model=SVC(kernel="linear").fit(X_train,y_train)
svm_model
# ylerimi cemer olarak ayrıstırmaya calisirsam "rbf" kullanılırım

KeyboardInterrupt: 

In [37]:
y_pred=svm_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.9207920792079208

In [38]:
print(classification_report(y_test,y_pred))
#default parametrelerle kurdumuz SVM modelimizin basarisi

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       554
           1       1.00      0.08      0.14        52

   micro avg       0.92      0.92      0.92       606
   macro avg       0.96      0.54      0.55       606
weighted avg       0.93      0.92      0.89       606



# 7.1-) SVM Model Tuning

In [5]:
svc_params={"C":np.arange(1,10)} #C=error ratelerdir yani bu error seviyesinde dur bilgisidi
svc=SVC(kernel="linear")
svc_cv_model=GridSearchCV(svc,svc_params,
                         cv=10,
                         n_jobs=-1,
                         verbose=2)


In [ ]:
svc_cv_model.fit(X_train,y_train)

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
#hocam bu svm tuningi 2 gündür en az 5 kere denedim ve hep bilgisayar dondu en sonunda ise kapandı 
#ve şirket bilgisayarı oldugu icin daha fazla zorlamak istemedim svm model tuningini yapamadan teslim etmek zorunda kaldım